# Something else that pybtls can do.

In pybtls, different lanes can use different vehicle generation models. 

There are four lanes of traffic: 

- Lane 1: Vehicles are generated from Grave model, and headways are in HeDS condition.
- Lane 2: Vehicles are generated from Nominal model, and headways are in Congested condition. 
- Lane 3: Vehicles are generated from Garage model, and headways are in Constant condition. 
- Lane 4: Vehicles are generated from Garage model, and headways are in Freeflow condition (Poisson arrival model). 

This time we only generate the traffic without effect calculation. 

---

Import the packages and initialize the traffic generator. 

In [15]:
import pybtls as pb
from pathlib import Path


traffic_gen = pb.TrafficGenerator(no_lane=4)


Set lane 1. 

In [16]:
# Define the flow composition
lfc_lane1 = pb.LaneFlowComposition(lane_index=1, lane_dir=1)
lfc_lane1.assign_lane_data(
    hourly_truck_flow=[100] * 24,
    hourly_car_flow=[0] * 24,
    hourly_speed_mean=[80 / 3.6 * 10] * 24,
    hourly_speed_std=[10.0] * 24,
    hourly_truck_composition=[
        [25.0, 25.0, 25.0, 25.0] for _ in range(24)
    ],  # only the Grave vehicle generator (gen by distribution) requires this info.
)

# Set the Grave vehicle generator and use the Auxerre truck features
vehicle_gen_Grave = pb.VehicleGenGrave(traffic_site="Auxerre")

# Set the headway generator
headway_gen_HeDS = pb.HeadwayGenHeDS()

# Config the first lane
traffic_gen.add_lane(
    vehicle_gen=vehicle_gen_Grave, headway_gen=headway_gen_HeDS, lfc=lfc_lane1
)


Set lane 2.

In [17]:
# Define the flow composition
lfc_lane2 = pb.LaneFlowComposition(lane_index=2, lane_dir=1)
lfc_lane2.assign_lane_data(hourly_truck_flow=[100] * 24, hourly_car_flow=[0] * 24)

# Set the Nominal vehicle generator with a custom vehicle
vehicle = pb.Vehicle(no_axles=3)
vehicle.set_axle_weights([100.0, 100.0, 100.0])  # in kN
vehicle.set_axle_spacings([3.0, 7.0, 0.0])  # in m
vehicle.set_axle_widths([2.0, 2.0, 2.0])  # in m

vehicle_gen_Nominal = pb.VehicleGenNominal(
    nominal_vehicle=vehicle, COV_list=[1.0, 1.0]
)

# Set the headway generator
headway_gen_Congested = pb.HeadwayGenCongested(
    congested_spacing=26.1, congested_speed=36.0, congested_gap_coef_var=0.05
)

# Config the second lane
traffic_gen.add_lane(
    vehicle_gen=vehicle_gen_Nominal, headway_gen=headway_gen_Congested, lfc=lfc_lane2
)


Set lane 3 and lane 4. 

In [18]:
# Set the flow composition
lfc_lane3 = pb.LaneFlowComposition(lane_index=3, lane_dir=2)
lfc_lane3.assign_lane_data(hourly_truck_flow=[100] * 24, hourly_car_flow=[0] * 24)

# Set the Garage vehicle generator
kernel = [[1.0, 0.08], [1.0, 0.05], [1.0, 0.02]]
vehicle_gen_Garage = pb.VehicleGenGarage(
    garage=Path(".") / "garage.txt", kernel=kernel, garage_format=4
)

# Set the headway generator
headway_gen_Constant = pb.HeadwayGenConstant(constant_speed=36.0, constant_gap=5.0)

# Config the third lane
traffic_gen.add_lane(
    vehicle_gen=vehicle_gen_Garage, headway_gen=headway_gen_Constant, lfc=lfc_lane3
)

# Set the flow composition
lfc_lane4 = pb.LaneFlowComposition(lane_index=4, lane_dir=2)
lfc_lane4.assign_lane_data(
    hourly_truck_flow=[100] * 24,
    hourly_car_flow=[0] * 24,
    hourly_speed_mean=[80 / 3.6 * 10] * 24,
    hourly_speed_std=[10.0] * 24,
)

# Use the same vehicle generator as the third lane.

# Set the headway generator
headway_gen_Freeflow = pb.HeadwayGenFreeflow()

# Config the fourth lane
traffic_gen.add_lane(
    vehicle_gen=vehicle_gen_Garage, headway_gen=headway_gen_Freeflow, lfc=lfc_lane4
)


Set the output to export the generated traffic.

In [19]:
output_config = pb.OutputConfig()
output_config.set_vehicle_file_output(
    write_vehicle_file=True,
    vehicle_file_format=4,
    vehicle_file_name="generated_traffic_file.txt",
)


Do the simulation and read the output. 

In [20]:
sim_task = pb.Simulation(Path(".") / "temp")
sim_task.add_sim(
    traffic=traffic_gen,
    no_day=30,  # 30 days traffic
    output_config=output_config,
    track_progress=True,  # optional, print the simulation days. 
    tag="Case4",
)

# Run simulation
sim_task.run(no_core=1)

# Get the output
case4_output = sim_task.get_output()["Case4"]
print(case4_output.get_summary())

from IPython.display import display
display(case4_output.read_data("traffic")["generated_traffic_file"])


Starting simulation...
Day complete...

Flushing buffer of 10000 vehicles at 1/1/0 5:13:46.5162

Flushing buffer of 10000 vehicles at 1/1/0 10:26:25.8773

Flushing buffer of 10000 vehicles at 1/1/0 15:39:30

Flushing buffer of 10000 vehicles at 1/1/0 20:51:52.9777

Flushing buffer of 10000 vehicles at 2/1/0 2:3:25

Flushing buffer of 10000 vehicles at 2/1/0 7:16:6.92188

Flushing buffer of 10000 vehicles at 2/1/0 12:30:14.9937

Flushing buffer of 10000 vehicles at 2/1/0 17:43:19.2472

Flushing buffer of 10000 vehicles at 2/1/0 22:54:45

Flushing buffer of 10000 vehicles at 3/1/0 4:6:24.3359

Flushing buffer of 10000 vehicles at 3/1/0 9:18:17.0485

Flushing buffer of 10000 vehicles at 3/1/0 14:28:27.5164

Flushing buffer of 10000 vehicles at 3/1/0 19:41:25

Flushing buffer of 10000 vehicles at 4/1/0 0:55:25

Flushing buffer of 10000 vehicles at 4/1/0 6:9:12.8006

Flushing buffer of 10000 vehicles at 4/1/0 11:21:45.6165

Flushing buffer of 10000 vehicles at 4/1/0 16:34:43.5363

Flushing 

,Head,Year,Month,Day,Hour,Min,Sec,NoAxles,NoAxleGroups,GVW,Velocity,Length,Lane,Dir,Trns,AxleWeights,AxleSpacings,AxleWidths
0,1001,1,1,0,0,0,3.689,3,0,371.25945,10.0,10.902,2,1,1.8,"[110.61756000000001, 185.69349000000003, 74.93...","[3.994, 6.908, 0.0]","[1.98, 1.98, 1.98]"
1,47799,1,1,0,0,0,5.000,2,2,15.48018,10.0,2.875,2,2,1.8,"[7.9461, 7.534080000000001]","[2.875, 0.0]","[1.98, 1.98]"
2,1001,1,1,0,0,0,7.367,3,0,330.67548,10.0,10.228,2,1,1.8,"[76.47876000000001, 135.17199000000002, 119.02...","[2.715, 7.513, 0.0]","[1.98, 1.98, 1.98]"
3,42741,1,1,0,0,0,10.000,2,2,15.06816,10.0,2.897,2,2,1.8,"[8.328690000000002, 6.739470000000001]","[2.897, 0.0]","[1.98, 1.98]"
4,1001,1,1,0,0,0,11.100,3,0,338.75892,10.0,12.807,2,1,1.8,"[51.01200000000001, 110.97072000000001, 176.77...","[2.71, 10.097, 0.0]","[1.98, 1.98, 1.98]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381671,42332,5,2,0,23,59,55.000,2,2,12.67452,10.0,2.744,2,2,1.8,"[6.729660000000001, 5.93505]","[2.744, 0.0]","[1.98, 1.98]"
1381672,1001,5,2,0,23,59,55.680,3,0,221.61771,10.0,11.796,2,1,1.8,"[55.142010000000006, 64.25550000000001, 102.2202]","[3.887, 7.91, 0.0]","[1.98, 1.98, 1.98]"
1381673,1001,5,2,0,23,59,59.413,3,0,333.21627,10.0,13.354,2,1,1.8,"[175.33413000000002, 122.13450000000002, 35.74...","[5.304, 8.05, 0.0]","[1.98, 1.98, 1.98]"
1381674,47792,6,2,0,0,0,0.000,3,2,122.55633,10.0,5.241,2,2,1.8,"[56.407500000000006, 36.31662, 29.84202]","[3.924, 1.317, 0.0]","[1.98, 1.98, 1.98]"
